In [ ]:
---
title: Responses of spring-mass-damper system
description: Forced response and initial transients
author: Daning H.
show-code: False
show-prompt: False
params:
    w0:
        input: numeric
        label: Natural frequency, omega_0
        value: 1.0
        step: 0.2
    zt:
        input: numeric
        label: Damping ratio, zeta
        value: 0.0
        step: 0.1
    y01:
        input: numeric
        label: Initial displacement
        value: 1.0
        step: 0.1
    y02:
        input: numeric
        label: Initial velocity
        value: 1.0
        step: 0.1
---

We visualize the response of a spring-mass-damper system, examine the components of the response, and connect these components to the concepts of forced response and initial transients in Laplace Transform.

The governing equation of the system is written as,
$$
y'' + 2\zeta\omega_0 y' + \omega_0^2 y = \sin(\omega t),\ y(0)=y_0,\ y'(0)=y'_0
$$
where $\omega_0$ is the natural frequency, $\zeta$ is damping ratio, $y$ is the displacement, and the right-hand side is a sinusoidal input with frequency $\omega$.  The transfer function is,
$$
Q(s) = \frac{1}{s^2 + 2\zeta\omega_0 s + \omega_0^2}
$$
The Laplace Transform of the input is
$$
R(s) = \frac{\omega}{s^2+\omega^2}
$$
The Laplace Transform of the output is
$$
Y(s) = \frac{\omega}{(s^2+\omega^2)(s^2 + 2\zeta\omega_0 s + \omega_0^2)} + \frac{(s+2\zeta\omega_0)y_0 + \omega_0^2 y'_0}{s^2 + 2\zeta\omega_0 s + \omega_0^2}
$$
where the two terms are the forced response and initial transients, respectively.

While looking at the interaction, let's fix $\omega_0=1$ and $\zeta=0$, pick any non-zero initial conditions you like, and sweep over different $\omega$'s.

+ The first plot shows the input, and the second shows the output; the 3rd and 4th show the forced response and initial transients, respectively.  Once the initial conditions are fixed, the initial transients are fixed and show persisting oscillation.  The frequency of forced response follows that of the input.  The output is just the sum of the two.
+ When the input frequency $\omega=\omega_0=1$, the forced response starts to grow unbounded.  This is when the resonance happens.

Next, we can pick a nonzero value of $\zeta$.
+ Does the initial transients persist?  Why?
+ For the same input frequency, do we get the same amplitude of output as the case of $\zeta=0$?
+ What are differences between the responses when $\zeta<1$ and $\zeta>1$?  (e.g., level of oscillation)

In [14]:
w0  = 1.0
zt  = 0.5
y01 = 1.0
y02 = 1.0

In [15]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from scipy import integrate as spi

# Generate data
def genA(ab):
    a, b = ab
    A = np.array([[0., 1.], [-b, -a]])
    return A

def genR(k):
    return lambda t, _k=k: np.sin(_k*t)

def dyn(t, z, ab, inp):
    A = genA(ab)
    f = np.array([0., inp(t)])
    dz = A.dot(z) + f
    return dz

# ODE parameters
a  = 2*zt*w0
b  = w0**2
if zt > 1:
    m, n = -zt*w0, np.sqrt(zt**2-1)*w0
    l1, l2 = m+n, m-n
else:
    re, im = -zt*w0, np.sqrt(1-zt**2)*w0
    l1, l2 = re+1j*im, re-1j*im

# Simulation setup
tf = 32.0
Nt = 201
Ns = 41
ts = np.linspace(0, tf, Nt)
ab = (a, b)
y0f = [0.0, 0.0]
y0i = [y01, y02]

# t-domain results
inp = []
res = []
fcd = []
ini = []
ss = np.linspace(0., 2, Ns)
for _s in ss:
    rt  = genR(_s)
    sol = spi.solve_ivp(dyn, [0,tf], y0i, t_eval=ts, args=(ab, rt))
    slf = spi.solve_ivp(dyn, [0,tf], y0f, t_eval=ts, args=(ab, rt))
    sli = spi.solve_ivp(dyn, [0,tf], y0i, t_eval=ts, args=(ab, lambda t: np.zeros_like(t)))
    inp.append(rt(ts))
    res.append(sol.y[0])
    fcd.append(slf.y[0])
    ini.append(sli.y[0])

fig = make_subplots(rows=4, cols=1,
                    vertical_spacing=0.05,
                    subplot_titles=("Input", "Output", "Forced responses", "Initial transients"))

# Variable plots
## Input
for _i in range(Ns):
    fig.add_trace(go.Scatter(
        visible=False, mode='lines', showlegend=False,
        line=dict(color="black", width=1),
        x=ts, y=inp[_i]),
        row=1, col=1)

## Output
for _i in range(Ns):
    fig.add_trace(go.Scatter(
        visible=False, mode='lines', showlegend=False,
        line=dict(color="black", width=1),
        x=ts, y=res[_i]),
        row=2, col=1)

## Output - forced
for _i in range(Ns):
    fig.add_trace(go.Scatter(
        visible=False, mode='lines', showlegend=False,
        line=dict(color="black", width=1),
        x=ts, y=fcd[_i]),
        row=3, col=1)

## Output - IC
for _i in range(Ns):
    fig.add_trace(go.Scatter(
        visible=False, mode='lines', showlegend=False,
        line=dict(color="black", width=1),
        x=ts, y=ini[_i]),
        row=4, col=1)

## Initial visibility
for _i in range(4):
    fig.data[_i*Ns].visible = True

# Create and add slider
steps = []
for _i in range(Ns):
    mask = [False] * (Ns*4)
    mask[_i] = True
    mask[_i+Ns] = True
    mask[_i+2*Ns] = True
    mask[_i+3*Ns] = True
    step = dict(
        method="update",
        args=[{"visible": mask}],
        label=f"{ss[_i]:3.2f}"
    )
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "omega = "},
    pad={"t": 50},
    steps=steps
)]

fig.update_xaxes(title_text='', row=1, col=1)
fig.update_yaxes(title_text='', range=[-1.05,1.05], row=1, col=1)
fig.update_xaxes(title_text='', row=2, col=1)
fig.update_yaxes(title_text='', range=[np.min(res), np.max(res)], row=2, col=1)
fig.update_xaxes(title_text='', row=3, col=1)
fig.update_yaxes(title_text='', range=[np.min(fcd), np.max(fcd)], row=3, col=1)
fig.update_xaxes(title_text='', row=4, col=1)
fig.update_yaxes(title_text='', range=[np.min(ini), np.max(ini)], row=4, col=1)

fig.update_layout(
    sliders=sliders,
    autosize=False,
    width=600,
    height=800)

fig.show()